<a href="https://colab.research.google.com/github/sabumjung/DL-test/blob/master/xgboost_and_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install library via pip

In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=c25ecd5d2a39889a978a9b84e46e0997fd63114d2bd8cea7f0752e1169763e0c
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


## Import required libraries

In [41]:
#Import libraries
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
import xgboost as xgb

## We define a function to run Bayesian optimization given data, function to optimize and its hyperparameters:

In [3]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 5
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

In [4]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (1, 150),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

In [5]:
def xgb_optimization(cv_splits, eval_set):
    def function(eta, gamma, max_depth):
            return cross_val_score(
                   xgb.XGBClassifier(
                       objective="binary:logistic",
                       learning_rate=max(eta, 0),
                       gamma=max(gamma, 0),
                       max_depth=int(max_depth),                                               
                       seed=42,
                       nthread=-1,
                       scale_pos_weight = len(y_train[y_train == 0])/
                                          len(y_train[y_train == 1])),  
                   X=X_train, 
                   y=y_train, 
                   cv=cv_splits,
                   scoring="roc_auc",
                   fit_params={
                        "early_stopping_rounds": 10, 
                        "eval_metric": "auc", 
                        "eval_set": eval_set},
                   n_jobs=-1).mean()

    parameters = {"eta": (0.001, 0.4),
                  "gamma": (0, 20),
                  "max_depth": (1, 2000)}
    
    return function, parameters

In [6]:
#Train model
def train(X_train, y_train, X_test, y_test, function, parameters):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             max_depth=int(max(params["max_depth"], 1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    
    return model

In [48]:
# Prepare the data.
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
X = cancer["data"]
y = cancer["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                            stratify = y,
                                        random_state = 42)

In [49]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [68]:
jj = rfc_optimization(10)
print(jj[1])
print(jj[0])

{'n_estimators': (10, 1000), 'max_depth': (1, 150), 'min_samples_split': (2, 10)}
<function rfc_optimization.<locals>.function at 0x7fc6ebd35320>


In [69]:
train(X_train, y_train, X_test, y_test, function=jj[0], parameters = jj[1])

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.9928   |  54.29    |  4.513    |  227.3    |
|  2        |  0.9935   |  62.39    |  8.704    |  579.6    |
|  3        |  0.9934   |  27.43    |  2.74     |  466.1    |
|  4        |  0.9932   |  111.6    |  5.606    |  128.2    |
|  5        |  0.9937   |  114.1    |  3.086    |  659.0    |
|  6        |  0.9935   |  69.42    |  5.763    |  759.4    |
|  7        |  0.9937   |  114.2    |  3.402    |  657.9    |
|  8        |  0.9937   |  149.8    |  8.915    |  997.9    |
|  9        |  0.9932   |  6.034    |  9.23     |  999.7    |
|  10       |  0.9935   |  148.5    |  2.237    |  891.4    |


RandomForestClassifier(class_weight='balanced', max_depth=114,
                       min_samples_split=3, n_estimators=659, n_jobs=-1,
                       random_state=42)

In [63]:
eval_set=[(X_test, y_test)]
eval_set

[(array([[1.275e+01, 1.670e+01, 8.251e+01, ..., 8.045e-02, 3.071e-01,
          8.557e-02],
         [1.799e+01, 2.066e+01, 1.178e+02, ..., 1.974e-01, 3.060e-01,
          8.503e-02],
         [1.496e+01, 1.910e+01, 9.703e+01, ..., 1.489e-01, 2.962e-01,
          8.472e-02],
         ...,
         [1.170e+01, 1.911e+01, 7.433e+01, ..., 5.741e-02, 3.487e-01,
          6.958e-02],
         [1.450e+01, 1.089e+01, 9.428e+01, ..., 1.221e-01, 2.889e-01,
          8.006e-02],
         [1.218e+01, 1.784e+01, 7.779e+01, ..., 5.882e-02, 2.227e-01,
          7.376e-02]]),
  array([1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
         1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
         1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
         0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
         1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
         0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 

In [64]:
jj1 = xgb_optimization(10, eval_set)  # cv_splits, eval_set

In [65]:
jj1[0]

<function __main__.xgb_optimization.<locals>.function>

In [66]:
jj1[1]

{'eta': (0.001, 0.4), 'gamma': (0, 20), 'max_depth': (1, 2000)}

In [67]:
train(X_train, y_train, X_test, y_test, function=jj1[0], parameters = jj1[1])

|   iter    |  target   |    eta    |   gamma   | max_depth |
-------------------------------------------------------------
|  1        |  0.9851   |  0.3287   |  4.998    |  1.024e+0 |
|  2        |  0.9901   |  0.3012   |  2.481    |  1.876e+0 |
|  3        |  0.9861   |  0.3177   |  7.451    |  1.684e+0 |
|  4        |  0.9813   |  0.3134   |  8.16     |  629.4    |
|  5        |  0.9482   |  0.007783 |  7.863    |  1.958e+0 |
|  6        |  0.992    |  0.1276   |  1.096    |  1.875e+0 |
|  7        |  0.9918   |  0.3692   |  0.04656  |  1.853e+0 |
|  8        |  0.9878   |  0.2563   |  1.176    |  1.82e+03 |
|  9        |  0.9805   |  0.33     |  18.13    |  1.788e+0 |
|  10       |  0.8972   |  0.001    |  20.0     |  1.838e+0 |


KeyError: ignored